In [1]:
import sys

In [2]:
#%pip install numpy pandas #does not work

In [3]:
#%pip install networkx pydot gdal==3.0.4 #does not work

In [4]:
import networkx as nx

## Graphs:

In [5]:
TN = nx.Graph()

In [6]:
TN.add_nodes_from([
    ('A',  {'pois': ['bus_stop']}), 
    ('B',  {'pois': ['observation_tower']}), 
    ('D',  {'pois': []}), 
    ('E',  {'pois': ['resting_area']}), 
    ('K',  {'pois': ['shelter', 'drinking_water']}), 
    ('L1', {'pois': ['beach']}), 
    ('L2', {'pois': ['beach', 'campsite']}),
    ('M',  {'pois': ['peak']}),
    ('N',  {'pois': ['parking']}),
    ('Q',  {'pois': []}),
    ('X',  {'pois': ['castle', 'restaurant']}),
    ('W1', {'pois': ['museum']}),
    ('W2', {'pois': ['railway_station']}),
    ('Z',  {'pois': ['tourist_information', 'visitor_centre']}),
])

In [7]:
TN.add_edges_from([
    ('A',  'D',  {'length': 1, 'difficulty': 'easy'}),
    ('D',  'E',  {'length': 1, 'difficulty': 'easy'}),
    ('E',  'B',  {'length': 1, 'difficulty': 'easy'}),
    ('E',  'K',  {'length': 3, 'difficulty': 'easy'}),
    ('K',  'L1', {'length': 2, 'difficulty': 'easy'}),
    ('K',  'L2', {'length': 2, 'difficulty': 'easy'}),
    ('L1', 'L2', {'length': 1, 'difficulty': 'easy'}),
    ('D',  'M',  {'length': 2, 'difficulty': 'hard'}),
    ('M',  'Q',  {'length': 2, 'difficulty': 'hard'}),
    ('K',  'Q',  {'length': 2, 'difficulty': 'hard'}),
    ('N',  'Q',  {'length': 1, 'difficulty': 'easy'}),
    ('K',  'X',  {'length': 2, 'difficulty': 'easy'}),
    ('Q',  'X',  {'length': 2, 'difficulty': 'easy'}),
    ('X',  'Z',  {'length': 2, 'difficulty': 'easy'}),
    ('Z',  'W1', {'length': 1, 'difficulty': 'easy'}),
    ('Z',  'W2', {'length': 2, 'difficulty': 'easy'}),
])

In [8]:
for n1,n2 in TN.edges:
    edgedata = TN.get_edge_data(n1,n2)
    #print(n1 + "-" + n2 + ": " + str(edgedata['length']) + " " + edgedata['difficulty'])
    print(f"{n1: <2}-{n2: <2}: {edgedata['length']} {edgedata['difficulty']}")

A -D : 1 easy
B -E : 1 easy
D -E : 1 easy
D -M : 2 hard
E -K : 3 easy
K -L1: 2 easy
K -L2: 2 easy
K -Q : 2 hard
K -X : 2 easy
L1-L2: 1 easy
M -Q : 2 hard
N -Q : 1 easy
Q -X : 2 easy
X -Z : 2 easy
W1-Z : 1 easy
W2-Z : 2 easy


In [9]:
TN2 = TN.copy()

In [10]:
TN2.add_edges_from([
    ('B',  'D',  {'length': 2, 'difficulty': 'easy'}),
    ('Q',  'K',  {'length': 2, 'difficulty': 'easy'}),
])

In [11]:
for n1,n2 in TN2.edges:
    edgedata = TN2.get_edge_data(n1,n2)
    #print(n1 + "-" + n2 + ": " + str(edgedata['length']) + " " + edgedata['difficulty'])
    print(f"{n1: <2}-{n2: <2}: {edgedata['length']} {edgedata['difficulty']}")

A -D : 1 easy
B -E : 1 easy
B -D : 2 easy
D -E : 1 easy
D -M : 2 hard
E -K : 3 easy
K -L1: 2 easy
K -L2: 2 easy
K -Q : 2 easy
K -X : 2 easy
L1-L2: 1 easy
M -Q : 2 hard
N -Q : 1 easy
Q -X : 2 easy
X -Z : 2 easy
W1-Z : 1 easy
W2-Z : 2 easy


In [12]:
nx.shortest_path(TN, 'N', 'L2', lambda n1,n2,ed: ed['length'])

['N', 'Q', 'K', 'L2']

In [13]:
TN_easy = nx.Graph([(n1,n2,ed) for n1,n2,ed in TN.edges(data=True) if ed['difficulty'] == 'easy'])

In [14]:
nx.shortest_path(TN_easy, 'N', 'L2', lambda n1,n2,ed: ed['length'])

['N', 'Q', 'X', 'K', 'L2']

## User profile:

In [15]:
tripRangeMinLength, tripRangeMaxLength = 5.0, 20.0

In [16]:
tripDifficulties = ['easy', 'hard']

## Start \& destination nodes:

In [34]:
startTN = {'A', 'N'} 

In [45]:
destTN = {'N', 'W2'}

## Index computation:

1. Assign the value of PoiDist as minp n to each node n in the network (at least to the
reachable nodes for P - this can also be done dynamically during further steps).

In [46]:
min_p = [(n,0) for n in TN.nodes()] # no POI preference implemented yet

In [47]:
min_p

[('A', 0),
 ('B', 0),
 ('D', 0),
 ('E', 0),
 ('K', 0),
 ('L1', 0),
 ('L2', 0),
 ('M', 0),
 ('N', 0),
 ('Q', 0),
 ('X', 0),
 ('W1', 0),
 ('W2', 0),
 ('Z', 0)]

2. Generate a cover of P-matching connecting trips CT Set in their increasing order
of length, by BFS (breadth-first search), utilizing the minp n values so that each
of these trips will be a direct connecting trip with an optional spike to the closest
preferred POI. If StartT N ∩ DestT N != 0  than it must be separately done for
each a ∈ StartT N ∩ DestT N towards DestT N \ {a}, so that no returning trips
are included.

In [48]:
CTSet = []
for sn in startTN:
    destTN2 = destTN.copy()
    destTN2.discard(sn)
    for dn in destTN2:
        for p in nx.all_simple_paths(TN, sn, dn):
            CTSet.append(p)

In [49]:
CTSet

[['N', 'Q', 'M', 'D', 'E', 'K', 'X', 'Z', 'W2'],
 ['N', 'Q', 'K', 'X', 'Z', 'W2'],
 ['N', 'Q', 'X', 'Z', 'W2'],
 ['A', 'D', 'E', 'K', 'Q', 'N'],
 ['A', 'D', 'E', 'K', 'X', 'Q', 'N'],
 ['A', 'D', 'M', 'Q', 'N'],
 ['A', 'D', 'E', 'K', 'Q', 'X', 'Z', 'W2'],
 ['A', 'D', 'E', 'K', 'X', 'Z', 'W2'],
 ['A', 'D', 'M', 'Q', 'K', 'X', 'Z', 'W2'],
 ['A', 'D', 'M', 'Q', 'X', 'Z', 'W2']]

3. Assign to each non-spike section contained by trips of CT Set the length of the
shortest trip of such (denoted by mintD s ). The value length(s)/mintD s will be the
candidate for the eventual w CTV I of that section.

4. Assign the value of TripDist as mint n and TripPoiDist as mint p n w.r.t. P to each
node n in the network (its reachable part for P) and similarly, mint s , mint p s for
sections s respectively. 12
12 This step can be done by generating an optimized cover of direct connecting trips DCT Set, regardless
of POI visits, and (optional) spikes connected to these trips from each node. Note that in some cases, the
two values mint n and mint p n will be based on different trips, and neither of them must be based on a direct
connecting trip in which node n appears. Furthermore, if retOnly p is true then this step must be done separately
for each a ∈ StartT N ∩ DestT N, storing possibly multiple values for each (a, n).5. Find and generate covering loops LT Set each having at least one section not yet
scored by w CTV I , in their increasing order of LoopLengthPoiDist, which is the
value mintL l = min({mint p k + length(l)|k ∈ nodes(l)} ∪ {mint m + length(l) +
minp n |m, n ∈ nodes(l)}. This is going to be the length of the trip giving the weight
value candidates for LTVI of the circle sections in the loop. Assign the value
mintL l to each loop l. 13
6. Assign to each circle section s contained by loops of LT Set the value of mintL l
by a containing loop for which it is minimal. The value length(s)/mintL l will be
the candidate for the eventual w LTV I of that section.
7. Remaining sections not covered yet by Steps 3 and 6 can only be parts of back-
and-forth sections (as parts of spikes or loops) of profile-relevant trips. Prune
them if poiTypes p is non-empty: remove sections (set their mint s , mint p s values
as ∞) not being part of any loops of Step 6 or spikes leading to a preferred POI.
8. Assign to each remaining section having finite mint p s the value minS s =
min({mint p s } ∪ {mintL l |s ∈ sections(l)}). The value length(s)/mintS s will be
the candidate for the eventual w STV I of that section.
9. Upscale the weights for short trips: if any value assigned to sections in Steps 3,
6, 8 is lower than min(tripRange p ), then set it explicitly to to min(tripRange p )
so that the weight will be the maximal possible value (c.f. MatchT PD).
10. Aggregate (sum by the respective trip variety index type CTV I, LTV I, STV I) and
output the computed section weight values w CTV I , w LT